In [150]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from lib import utils
import os
from binance.spot import Spot
from dotenv import load_dotenv
load_dotenv()

True

In [151]:
a = {"a":{"b": "b", "c":"c"}, "d":"d"}
a.update({"e":"e"})
print(a | {"f":"f"} | {"a":{"f":"f"}})
a.update({"a":{"f":"f"}})
a

{'a': {'f': 'f'}, 'd': 'd', 'e': 'e', 'f': 'f'}


{'a': {'f': 'f'}, 'd': 'd', 'e': 'e'}

In [152]:
bot = {
    "prices": {"updated": 0, "data": {}},
    "precisions": {"updated": 0, "data": {}}
}

In [171]:
user = {
    "updated": 0,
    "updated_date": utils.current_readable_time(),
    "auth": {
        "updated": 0,
        "data": {
            "username": "root",
            "email": "root@example.com",
            "password_hash": '',
            "binance_api_key": '',
            "binance_secret_hash": ''
            }
    },
    "details": {
        "updated": 0,
        "data": {
            "active": True,
            "chat_id": 1031182213
            }
    },
    "account": {           
        "updated": 0,
        "data": {
            "leverage": 5,
            "value_USDT": 0,
            "value_BTC": 0,
            # "levered_ratio": np.sum(grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"]) / total_balance,
            # "unlevered_ratio": np.sum(grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"]) / total_balance,
            "collateral_margin_level": 0,
            "collateral_value_USDT": 0
        }
    },
    # "leaders": {"3846188874749232129": 1, "3842534998056366337":1},
    "leaders": {
        "updated": 0,
        "data": {
            "3846188874749232129": 1,
            "3907342150781504256":1
            }
        },
    "positions":{
        "updated": 0,
        "data": []
    },
    "mix": {
        "updated": 0,
        "data": {'symbol': {}, 'BAG': {}}
    }
}

BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_SECRET_KEY = os.environ.get("BINANCE_SECRET_KEY")

client = Spot(api_key=BINANCE_API_KEY, api_secret=BINANCE_SECRET_KEY)

partial_update = ['auth', 'details', 'account']

async def database_update(obj: object, update: object, collection: str) -> bool:
    current_time = utils.current_time()

    obj["updated"] = current_time
    obj["updated_date"] = utils.current_readable_time()
    for category_key in update.keys():
        category_obj = obj[category_key]
        category_obj["updated"] = current_time
        category_data = category_obj["data"]
        
        if category_key in partial_update:
            category_data.update(update[category_key])
        else:
            obj[category_key]["data"] = update[category_key]
    
    
    # self.app.db[collection].update_one({"_id": obj["_id"]}, {"$set": update})

    return True

def user_account_update(user, symbol_prices): #self, user
    weigth = 10

    margin_account_data = client.margin_account()

    positions = []
    for asset in margin_account_data["userAssets"]:
        amount = float(asset["netAsset"])
        if amount != 0 and asset["asset"] != 'USDT':
            positions.append(asset)
    # positions_df = pd.DataFrame(positions)
    # positions_df["SYMBOL"] = positions_df["asset"] + 'USDT' 
    positions = pd.DataFrame(positions)
    positions["symbol"] = positions["asset"] + 'USDT'
    positions = positions.merge(symbol_prices, on='symbol', how='left')
    positions["LEVERED_VALUE"] = positions["netAsset"] * positions['SYMBOL_PRICE']
    positions["UNLEVERED_VALUE"] = positions["LEVERED_VALUE"] / user["account"]["data"]["leverage"]
    positions["ABSOLUTE_LEVERED_VALUE"] = abs(positions["LEVERED_VALUE"])
    positions["ABSOLUTE_UNLEVERED_VALUE"] = abs(positions["UNLEVERED_VALUE"])

    assetBTC = float(margin_account_data["totalNetAssetOfBtc"])
    valueUSDT = float(client.ticker_price("BTCUSDT")["price"]) * assetBTC
    levered_ratio = positions["ABSOLUTE_LEVERED_VALUE"].sum() / valueUSDT
    unlevered_ratio = positions["ABSOLUTE_UNLEVERED_VALUE"].sum() / valueUSDT

    user_account_update = {
        "account": {
            "leverage": 5,
            "value_BTC": assetBTC,
            "value_USDT": valueUSDT,
            "levered_ratio": levered_ratio,
            "unlevered_ratio": unlevered_ratio,
            "collateral_margin_level": float(margin_account_data["totalCollateralValueInUSDT"]),
            "collateral_value_USDT": float(margin_account_data["collateralMarginLevel"])
        },
        "positions": positions.to_dict(),
    }

    return user_account_update

# user_account = user_account_update(user)
# user_account_update_success = await database_update(user, user_account, 'users')
# user

In [155]:
GATEWAY_HOST = 'https://www.binance.com'
API_PATH = '/'.join([GATEWAY_HOST, 'bapi/futures/v1'])

endpoints = {
    "positions" : {
        "path": 'public/future/copy-trade/lead-data/positions?portfolioId=%s', 
        "type": "simple",
        "params": {}
        },
    "performance" : {
        "path": 'public/future/copy-trade/lead-portfolio/performance?portfolioId=%s&timeRange=%s',
        "type": "simple",
        "params": {"timeRange": "90D"}
        },
    "detail" : {
        "path": 'friendly/future/copy-trade/lead-portfolio/detail?portfolioId=%s', 
        "type": "simple",
        "params": {}
    },
    "chart" : {
        "path": 'public/future/copy-trade/lead-portfolio/chart-data?portfolioId=%s&timeRange=%s&dataType=%s', 
        "type": "simple",
        "params": {"timeRange": "90D", "dataType": "ROI"}
    },
    "position_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/position-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    },
    "transfer_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/transfer-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    }
}


async def fetch_data(leaderId, endpointType, params={}):
    endpoint = endpoints[endpointType]
    # Filter out the empty params
    filtered_params = {}
    for default_key, default_value in endpoint['params'].items():
        if default_key in params.keys() and params[default_key] is not None:
            filtered_params[default_key] = params[default_key]
        else:
            filtered_params[default_key] = default_value

    if endpoint["type"] == 'simple':
        # Interpolate strings
        path = endpoint["path"] % (leaderId, *filtered_params.values())
        url = '/'.join([API_PATH, path])

        response = requests.get(url)

    if endpoint["type"] == 'paginated':
        url = '/'.join([API_PATH, endpoint["path"]])

        response = requests.post(
            url,
            json={"portfolioId": leaderId} | filtered_params,
            )

    return response.json()


async def fetch_pages(leaderId, endpointType, params={}, page_number=None, result=None, latest_item=None, reference=None, progress_bar=None):
    if page_number is None:
        page_number = 1
    if result is None:
        result = []

    response = await fetch_data(leaderId, endpointType, {"pageNumber": page_number} | params)

    if response["success"]:
        response_data = response["data"]
        response_list = response_data["list"]

        if latest_item and reference:
            response_list = sorted(response_list, key=lambda x: x[reference], reverse=True)
            item_index = 0

            for item in response_list:
                if item[reference] > latest_item[reference]:
                    result.append(item)
                    item_index += 1

                    if item_index == 10:
                        next_page = page_number + 1
                        return await fetch_pages(leaderId, endpointType, params, next_page, result, latest_item, reference)
                else:
                    # print({ "success": True, "reason": "partial", "message": f"Fetched pages {endpointType} - finished by update", "data": result })
                    return result
        else:
            result += response_list
            total_n_results = response_data["total"]
            pages_length = total_n_results // 10

            if progress_bar is None:
                progress_bar = tqdm(total=total_n_results)

            progress_bar.update(len(response_list))
            # If remainder, add another page
            if total_n_results % 10:
                pages_length += 1

            if page_number <= pages_length:
                next_page = page_number + 1

                return await fetch_pages(leaderId, endpointType, params, next_page, result, progress_bar=progress_bar)
            else:
                return result
    
    else:
        return { "success": False, "message": f"Could not fetch page {page_number}/{pages_length} of {endpointType}" }
    

In [156]:
async def leader_details_update(leader=None, binance_id:str=None):
    if leader:
        binance_id = leader["details"]["data"]["leadPortfolioId"]

    details = await fetch_data(binance_id, 'detail')
    leader_details = details["data"]

    leader_details_update = {
        "details":  leader_details
    }

    return leader_details_update


async def leader_performance_update(leader):
    binance_id = leader["details"]["data"]["leadPortfolioId"]

    performance = await fetch_data(binance_id, 'performance')
    leader_performance = performance["data"]

    leader_performance_update = {
        "performance": leader_performance
    }

    return leader_performance_update


async def leader_account_update(leader):
    binance_id = leader["details"]["data"]["leadPortfolioId"]
    
    transfer_history = await fetch_pages(binance_id, 'transfer_history')

    leader_transfer_history = pd.DataFrame(transfer_history)
    leader_transfer_history["LEADER_ID"] = binance_id
    leader_transfer_history["INCOMING"] = leader_transfer_history["to"] == 'Lead Trading Account'
    leader_transfer_history["RELATIVE_VALUE"] = leader_transfer_history.apply(lambda row: row["amount"] if row["INCOMING"] else -row["amount"], axis=1)

    position_history = await fetch_pages(binance_id, 'position_history')
    leader_position_history = pd.DataFrame(position_history)
    leader_position_history["LEADER_ID"] = binance_id

    transfer_balance = leader_transfer_history["RELATIVE_VALUE"].sum()
    historic_PNL = leader_position_history["closingPnl"].sum()
    total_balance = transfer_balance + historic_PNL

    leader_account_update = {
        "account": {
            "transfer_balance": transfer_balance,
            "historic_PNL": historic_PNL,
            "total_balance": total_balance,
        },
    }

    return leader_account_update


sum_columns = [
    "positionAmount",
    "unrealizedProfit",
    "cumRealized",
    "notionalValue",
	"markPrice",
    "ABSOLUTE_LEVERED_VALUE",
    "ABSOLUTE_UNLEVERED_VALUE"
]

average_columns = [
    "entryPrice",
	"markPrice",
	"leverage",
	"breakEvenPrice",
]

drop_columns = [
    "id",
    "collateral",
    "isolated",
    "isolatedWallet",
    "adl",
    "askNotional",
    "bidNotional"
]


def aggregate_leader_positions(group: pd.DataFrame) -> pd.Series:
    result = {}
    
    for column in sum_columns: result[column] = group[column].sum()
    for column in average_columns: result[column] = np.average(group[column], weights=group["positionAmount"])

    if len(group) > 1: result["positionSide"] = "BOTH"
    else: result["positionSide"] = group["positionSide"].values[0]

    # result["LEADER_IDS"] = list(set(group["LEADER_ID"]))
    return pd.Series(result)


async def leader_positions_update(leader, symbol_prices):
    binance_id = leader["details"]["data"]["leadPortfolioId"]

    positions = await fetch_data(binance_id, 'positions')
    leader_positions = pd.DataFrame(positions["data"])
    leader_positions["LEADER_ID"] = binance_id

    filtered_leader_positions = leader_positions.apply(lambda column: column.astype(float) if column.name in sum_columns + average_columns else column)
    filtered_leader_positions = filtered_leader_positions.loc[(filtered_leader_positions["positionAmount"] != 0) | (filtered_leader_positions["collateral"] != "USDT")]
    filtered_leader_positions = filtered_leader_positions.drop(columns=drop_columns)
    filtered_leader_positions["ABSOLUTE_LEVERED_VALUE"] = abs(filtered_leader_positions["notionalValue"])
    filtered_leader_positions["ABSOLUTE_UNLEVERED_VALUE"] = filtered_leader_positions["ABSOLUTE_LEVERED_VALUE"] / filtered_leader_positions["leverage"]

    grouped_leader_positions = filtered_leader_positions.groupby("symbol").apply(aggregate_leader_positions, include_groups=False).reset_index()
    grouped_leader_positions = grouped_leader_positions.rename(columns={key: key + "_SUM" for key in sum_columns} | {key: key + "_AVERAGE" for key in average_columns})
    grouped_leader_positions["LEVERED_POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"] / grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"].sum()
    grouped_leader_positions["UNLEVERED_POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"] / grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"].sum()

    total_balance = leader["account"]["data"]["total_balance"]
    levered_ratio = grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"].sum() / total_balance
    unlevered_ratio = grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"].sum() / total_balance

    leader_mix = grouped_leader_positions[['symbol', 'positionAmount_SUM']].rename(columns={"positionAmount_SUM": "BAG"})

    leader_positions_update = {
        "account": {
           "levered_ratio": levered_ratio,
            "unlevered_ratio": unlevered_ratio,
        },
        "positions": filtered_leader_positions.to_dict(),
        "grouped_positions": grouped_leader_positions.to_dict(),
        "mix": leader_mix.to_dict()
    }

    leader_symbol_prices = grouped_leader_positions[["symbol", "markPrice_AVERAGE"]].rename(columns={"markPrice_AVERAGE": 'SYMBOL_PRICE'})
    symbol_prices = pd.concat([symbol_prices, leader_symbol_prices]).groupby('symbol').agg('mean') if symbol_prices.size > 0 else leader_symbol_prices

    return leader_positions_update, symbol_prices

In [157]:
pool = {}

In [173]:

async def get_leader(symbol_prices, leader=None, binance_id:str=None):
        if leader:
            # leader = self.app.db.leaders.find_one({"_id": leader_id})
            leader_details = await leader_details_update(leader=leader)

        if binance_id:
            leader = {
                "details":{
                    "data":{}
                 },
                "account":{
                    "data":{}
                 },
                "positions":{
                    "data":[]
                 },
                "grouped_positions":{
                    "data":[]
                 },
                "mix":{
                    "data":[]
                 },
                "performance":{
                    "data":{}
                 },
            }
            leader_details = await leader_details_update(binance_id=binance_id)
        
        if leader:
            leader_details_update_success = await database_update(obj=leader, update=leader_details, collection='leaders')
            
            leader_performance = await leader_performance_update(leader)
            leader_performance_update_success = await database_update(obj=leader, update=leader_performance, collection='leaders')
            
            leader_account = await leader_account_update(leader)
            leader_account_update_success = await database_update(obj=leader, update=leader_account, collection='leaders')

            leader_positions, symbol_prices = await leader_positions_update(leader, symbol_prices)
            leader_positions_update_success = await database_update(obj=leader, update=leader_positions, collection='leaders')

        return leader, symbol_prices


# users = self.app.db.users.find()
users = [user]
symbol_prices = pd.DataFrame(columns=['symbol', 'PRICE'])

for user in users:
    user_leaders = user["leaders"]["data"]
    if len(user_leaders) > 0:
        user_leader_mixes = pd.DataFrame(columns=['symbol', 'BAG'])

        for leader_id, weight in user_leaders.items():
            if leader_id not in pool.keys():
                leader, symbol_prices = await get_leader(symbol_prices, binance_id=leader_id) #* in prod should be leader
                pool[leader_id] = leader
            
            leader = pool[leader_id]
            user_leader_mix = pd.DataFrame(leader["mix"]["data"])
            user_leader_mix["BAG"] = user_leader_mix["BAG"] * weight
            user_leader_mixes = pd.concat([user_leader_mixes, user_leader_mix]) if user_leader_mixes.size > 0 else user_leader_mix
        
        user_mix = pd.DataFrame(user["mix"]["data"])
        # user_mix = pd.DataFrame([["ETHUSDT", 10], ["BTCUSDT", 10]], columns=['symbol', 'BAG'])
        # user_leader_mixes = pd.DataFrame([["ETHUSDT", 10], ["SAGAUSDT", 10]], columns=['symbol', 'BAG'])
        new_user_mix = user_leader_mixes.groupby('symbol').agg('sum').reset_index()

        if user_mix.to_dict() != new_user_mix.to_dict():
            user_account = user_account_update(user, symbol_prices)
            user_account_update_success = await database_update(obj=user, update=user_account, collection='users')

            if user_account_update_success:
                user_positions = pd.DataFrame(user["positions"]["data"])
        #         user_positions = pd.DataFrame(user["positions"]["data"])
            # new_user_mix["NEW"] = ~new_user_mix["symbol"].isin(user_mix["symbol"].unique())
                # build_user_mix(user, leader, weight)


new_user_mix   
        # diff_user_mix(user, roster)

    # if leader_id not in roster.keys():
    #     roster[leader_id] = leader
    # return roster[leader_id]

# leader_positions.head()

   asset        free locked borrowed interest    netAsset     symbol
0  MANTA  0.00741153      0        0        0  0.00741153  MANTAUSDT
   asset        free locked borrowed interest    netAsset     symbol PRICE
0  MANTA  0.00741153      0        0        0  0.00741153  MANTAUSDT   NaN


,symbol,BAG
0,CFXUSDT,874.000
1,ETHUSDT,43.167
2,SOLUSDT,1.000


In [162]:
user_positions

,asset,free,locked,borrowed,interest,netAsset,SYMBOL,symbol,PRICE,LEVERED_VALUE,UNLEVERED_VALUE,ABSOLUTE_LEVERED_VALUE,ABSOLUTE_UNLEVERED_VALUE


In [ ]:
leader_mix]) if user_leader_mixes.size > 0 else user

In [ ]:
leader_mix
        
        user_mix = pd.DataFrame(user["mix"]["data"])
        # user_mix = pd.DataFrame([["ETHUSDT", 10], ["BTCUSDT", 10]], columns=['symbol', 'BAG'])
        # user_leader_mixes = pd.DataFrame([["ETHUSDT", 10], ["SAGAUSDT", 10]], columns=['symbol', 'BAG'])
        new_user_mix = user_leader_mixes.groupby('symbol').agg('sum').reset_index()

        if user_mix.to_dict() != new_user_mix.to_dict():
            user_account = user_account_update(user)
            user_account_update_success = await database_update(obj=user, update=user_account, collection='users')

            if user_account_update_success:
                user_positions = pd.DataFrame(user["positions"]["data"])
        #         user_positions = pd.DataFrame(user["positions"]["data"])
            # new_user_mix["NEW"] = ~new_user_mix["symbol"].isin(user_mix["symbol"].unique())
                # build_user_mix(user, leader, weight)


new_user_mix   
        # diff_user_mix(user, roster)

    # if leader_id not in roster.keys():
    #     roster[leader_id] = leader
    # return roster[leader_id]

# leader_positions.head()

In [ ]:
user_positions
# user_mix
# pd.DataFrame(user["positions"]["data"])

,asset,free,locked,borrowed,interest,netAsset,SYMBOL
0,MANTA,0.00741153,0,0,0,0.00741153,MANTAUSDT


In [ ]:
# def update_user_mix(user, leader, weight):

leader_mix_df = pd.DataFrame(leader["mix"]["data"])
leader_mix_df["BAG"] = leader_mix_df["BAG"] * 3
leader_mix_df


,symbol,BAG
0,CFXUSDT,2622.000
1,ETHUSDT,-0.255
2,SOLUSDT,3.000


In [ ]:
# pd.DataFrame([["ETHUSDT", 10], ["BTCUSDT", 10]], columns=['symbol', 'BAG']).to_dict()

{'symbol': {0: 'ETHUSDT', 1: 'BTCUSDT'}, 'BAG': {0: 10, 1: 10}}

In [ ]:
user_mix_df = pd.DataFrame(user["mix"]["data"])
user_mix_df = pd.concat([user_mix_df, leader_mix_df], join='inner').reindex(leader_mix_df.index)

user_mix_df


/var/folders/1h/861cc3q97gb3f6r361ydtk9m0000gn/T/ipykernel_61643/574840544.py:2: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  user_mix_df = pd.concat([user_mix_df, leader_mix_df], join='inner').reindex(leader_mix_df.index)


,symbol,BAG
0,CFXUSDT,2622.000
1,ETHUSDT,-0.255
2,SOLUSDT,3.000


In [ ]:
leader_mix_df_2 = pd.DataFrame([["ETHUSDT", 10], ["BTCUSDT", 10]], columns=['symbol', 'BAG']).set_index('symbol')
user_mix_df = pd.concat([user_mix_df, leader_mix_df_2], join='inner').reindex(leader_mix_df.index)

pd.DataFrame(leader_mix_df_2.to_dict())

,BAG
ETHUSDT,10
BTCUSDT,10


In [ ]:
leader_mix_df_3 = pd.DataFrame({'BAG': {'ETHUSDT': 10, 'BTCUSDT': 10}})
leader_mix_df_3

,BAG
ETHUSDT,10
BTCUSDT,10


In [ ]:
user

{'updated': 1712843212719,
 'updated_date': '2024-04-11 15:46:52',
 'auth': {'updated': 0,
  'data': {'username': 'root',
   'email': 'root@example.com',
   'password_hash': '',
   'binance_api_key': '',
   'binance_secret_hash': ''}},
 'details': {'updated': 0, 'data': {'active': True, 'chat_id': 1031182213}},
 'account': {'updated': 1712843212719,
  'data': {'leverage': 5,
   'value_USDT': 2007.0507256056003,
   'value_BTC': 0.02871378,
   'collateral_margin_level': 2008.42726597,
   'collateral_value_USDT': 999.0,
   'levered_ratio': None,
   'unlevered_ratio': None}},
 'leaders': {'updated': 0,
  'data': {'3846188874749232129': 1, '3907342150781504256': 1}},
 'positions': {'updated': 1712843212719,
  'data': [{'asset': 'MANTA',
    'free': '0.00741153',
    'locked': '0',
    'borrowed': '0',
    'interest': '0',
    'netAsset': '0.00741153'}]},
 'mix': {'updated': 0, 'data': {'symbol': {}, 'BAG': {}}}}

In [ ]:
leader

{'details': {'data': {'userId': 840012258,
   'leadOwner': False,
   'hasCopy': False,
   'leadPortfolioId': '3907342150781504256',
   'nickname': '丹尼尔-彩',
   'nicknameTranslate': None,
   'avatarUrl': 'https://public.nftstatic.com/static/nft/res/37d50e09b3cd4489a5fd26426f72b99d.png',
   'status': 'ACTIVE',
   'description': '跟单阅读❗️❗️定额仓位跟单5%❗️❗️杠杠10X❗️❗️最低1000U起跟单，保持耐心🐯执行量化➕手工策略🐯交流V：w317540',
   'descTranslate': None,
   'favoriteCount': 2011,
   'currentCopyCount': 200,
   'maxCopyCount': 200,
   'totalCopyCount': 769,
   'marginBalance': '2410.94935999',
   'initInvestAsset': 'USDT',
   'futuresType': 'UM',
   'aumAmount': '731391.56527193',
   'copierPnl': '330562.62779289',
   'copierPnlAsset': 'USDT',
   'profitSharingRate': '10.00',
   'unrealizedProfitShareAmount': '0',
   'startTime': 1710854141075,
   'endTime': None,
   'closedTime': None,
   'tag': ['MEDIUM_LEVERAGE', 'API_KEY_TRADE'],
   'positionShow': True,
   'mockCopyCount': 836,
   'sharpRatio': None,
   'hasMock': Fa

In [ ]:
ero

NameError: name 'ero' is not defined

In [ ]:
for leader_id, weight in user["leaders"]["data"].items():
    # details = await fetch_data(leader_id, 'detail')
    # leader_details = details["data"]
    
    positions = await fetch_data(leader_id, 'positions')
    leader_positions = pd.DataFrame(positions["data"])
    leader_positions["LEADER_ID"] = leader_id
    
    # position_history = await fetch_pages(leader_id, 'position_history')
    # leader_position_history = pd.DataFrame(position_history)
    # leader_position_history["LEADER_ID"] = leader_id

    # transfer_history = await fetch_pages(leader_id, 'transfer_history')
    # leader_transfer_history = pd.DataFrame(transfer_history)
    # leader_transfer_history["LEADER_ID"] = leader_id
    # leader_transfer_history["INCOMING"] = leader_transfer_history["to"] == 'Lead Trading Account'
    # leader_transfer_history["RELATIVE_VALUE"] = leader_transfer_history.apply(lambda row: row["amount"] if row["INCOMING"] else -row["amount"], axis=1)
    
    # performance = await fetch_data(leader_id, 'performance')
    # leader_performance = performance["data"]

In [ ]:
leader_transfer_history["INCOMING"] = leader_transfer_history["to"] == 'Lead Trading Account'
leader_transfer_history["RELATIVE_VALUE"] = leader_transfer_history.apply(lambda row: row["amount"] if row["INCOMING"] else -row["amount"], axis=1)

leader_transfer_history.head()

,time,coin,amount,from,to,transType,LEADER_ID,INCOMING,RELATIVE_VALUE
0,1711015734693,USDT,62523.139709,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,62523.139709
1,1710910426929,USDT,2109.074057,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,2109.074057
2,1710631903556,USDT,48177.912951,Lead Trading Account,Fiat and Spot,LEAD_WITHDRAW,3846188874749232129,False,-48177.912951
3,1710631606472,USDT,48177.912951,Fiat and Spot,Lead Trading Account,LEAD_DEPOSIT,3846188874749232129,True,48177.912951
4,1710631512228,USDT,48175.731055,Lead Trading Account,Fiat and Spot,LEAD_WITHDRAW,3846188874749232129,False,-48175.731055


In [ ]:
leader_position_history

,id,symbol,type,opened,closed,avgCost,avgClosePrice,closingPnl,maxOpenInterest,closedVolume,isolated,side,status,updateTime,LEADER_ID
0,28831399,BCHUSDT,UM,1712752315689,1.712785e+12,603.280501,618.462375,-945.041280,62.248,62.248,Isolated,Short,All Closed,1712784529622,3846188874749232129
1,28675086,BCHUSDT,UM,1712711525798,1.712729e+12,666.038565,626.704920,2201.700750,55.975,55.975,Isolated,Short,All Closed,1712728550720,3846188874749232129
2,28368782,ETHUSDT,UM,1712627977755,NaN,3603.292522,3490.719604,-6972.078457,45.070,44.950,Isolated,Long,Partially Closed,1712784543076,3846188874749232129
3,27731385,BCHUSDT,UM,1712373128871,1.712461e+12,702.900561,684.506554,905.592130,49.233,49.233,Isolated,Short,All Closed,1712461052390,3846188874749232129
4,27604253,ETHUSDT,UM,1712319921263,1.712628e+12,3296.424227,3565.913762,16574.414869,41.217,61.503,Isolated,Long,All Closed,1712665128410,3846188874749232129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10433832,ETHUSDT,UM,1707831600369,1.707834e+12,2641.025157,2610.031900,-2356.820250,76.043,76.043,Isolated,Long,All Closed,1707834104910,3846188874749232129
96,10225346,ETHUSDT,UM,1707732084449,1.707831e+12,2483.687340,2576.053026,20238.614850,146.525,219.114,Isolated,Long,All Closed,1707831450716,3846188874749232129
97,9987905,ETHUSDT,UM,1707565217287,1.707732e+12,2479.567046,2473.923819,-832.652429,147.549,147.549,Isolated,Long,All Closed,1707731638248,3846188874749232129
98,9450605,ETHUSDT,UM,1707209329727,1.707565e+12,2334.502083,2476.586570,19930.475049,140.272,140.272,Cross,Long,All Closed,1707565060277,3846188874749232129


In [ ]:
leader_positions

,id,symbol,collateral,positionAmount,entryPrice,unrealizedProfit,cumRealized,askNotional,bidNotional,notionalValue,markPrice,leverage,isolated,isolatedWallet,adl,positionSide,breakEvenPrice,LEADER_ID
0,0_INJUSDT_BOTH,INJUSDT,USDT,0.0,0.0,0.00000000,0,0,0,0,32.16500000,10,False,0,0,BOTH,0.0,3907342150781504256
1,0_INJUSDT_LONG,INJUSDT,USDT,0.0,0.0,0.00000000,0,0,0,0,32.16500000,10,False,0,0,LONG,0.0,3907342150781504256
2,0_INJUSDT_SHORT,INJUSDT,USDT,0.0,0.0,0.00000000,0.85269999,0,0,0,32.16500000,10,False,0,0,SHORT,0.0,3907342150781504256
3,0_BNBUSDT_BOTH,BNBUSDT,USDT,0.00,0.0,0.00000000,0,0,0,0,602.58000000,10,False,0,0,BOTH,0.0,3907342150781504256
4,0_BNBUSDT_LONG,BNBUSDT,USDT,0.00,0.0,0.00000000,29.07310000,0,0,0,602.58000000,10,False,0,0,LONG,0.0,3907342150781504256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0_UNIUSDT_LONG,UNIUSDT,USDT,0,0.0,0.00000000,16.44899995,0,0,0,9.11064235,10,False,0,0,LONG,0.0,3907342150781504256
86,0_UNIUSDT_SHORT,UNIUSDT,USDT,0,0.0,0.00000000,769.38600013,0,0,0,9.11064235,10,False,0,0,SHORT,0.0,3907342150781504256
87,0_BTCUSDT_BOTH,BTCUSDT,USDT,0.000,0.0,0.00000000,0,0,0,0,70041.20000000,10,False,0,0,BOTH,0.0,3907342150781504256
88,0_BTCUSDT_LONG,BTCUSDT,USDT,0.000,0.0,0.00000000,50.85700000,0,0,0,70041.20000000,10,False,0,0,LONG,0.0,3907342150781504256


In [ ]:
filtered_leader_positions = leader_positions.apply(lambda column: column.astype(float) if column.name in sum_columns + average_columns else column)
filtered_leader_positions = filtered_leader_positions.loc[(filtered_leader_positions["positionAmount"] != 0) | (filtered_leader_positions["collateral"] != "USDT")]
filtered_leader_positions = filtered_leader_positions.drop(columns=drop_columns)
filtered_leader_positions["ABSOLUTE_LEVERED_VALUE"] = abs(filtered_leader_positions["notionalValue"])
filtered_leader_positions["ABSOLUTE_UNLEVERED_VALUE"] = filtered_leader_positions["ABSOLUTE_LEVERED_VALUE"] / filtered_leader_positions["leverage"]
filtered_leader_positions

,symbol,positionAmount,entryPrice,unrealizedProfit,cumRealized,notionalValue,markPrice,leverage,positionSide,breakEvenPrice,LEADER_ID,ABSOLUTE_LEVERED_VALUE,ABSOLUTE_UNLEVERED_VALUE
40,SOLUSDT,1.000,173.542500,0.767267,-317.41150,174.309767,174.309767,10.0,LONG,173.703363,3907342150781504256,174.309767,17.430977
44,ETHUSDT,-0.085,3519.670000,1.072634,5497.49111,-298.099316,3507.050780,10.0,SHORT,3517.910165,3907342150781504256,298.099316,29.809932
49,CFXUSDT,874.000,0.343479,1.288879,16.29431,301.489918,0.344954,10.0,LONG,0.343651,3907342150781504256,301.489918,30.148992


In [ ]:

def aggregate_positions(group: pd.DataFrame) -> pd.Series:
    result = {}
    
    for column in sum_columns: result[column] = group[column].sum()
    for column in average_columns: result[column] = np.average(group[column], weights=group["positionAmount"])

    if len(group) > 1: result["positionSide"] = "BOTH"
    else: result["positionSide"] = group["positionSide"].values[0]

    # result["LEADER_IDS"] = list(set(group["LEADER_ID"]))
    return pd.Series(result)

grouped_leader_positions = filtered_leader_positions.groupby("symbol").apply(aggregate_positions, include_groups=False)
grouped_leader_positions = grouped_leader_positions.rename(columns={key: key + "_SUM" for key in sum_columns} | {key: key + "_AVERAGE" for key in average_columns})
grouped_leader_positions["LEVERED_POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"] / grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"].sum()
grouped_leader_positions["UNLEVERED_POSITION_SHARE"] = grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"] / grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"].sum()

grouped_leader_positions.reset_index(inplace=True)

In [ ]:
leader_symbol_prices = grouped_leader_positions[["symbol", "markPrice_AVERAGE"]].rename(columns={"markPrice_AVERAGE": 'PRICE'})
leader_symbol_prices = leader_symbol_prices.groupby('symbol').agg('mean')
leader_symbol_prices


,PRICE
symbol,
CFXUSDT,0.338601
ETHUSDT,3508.867856
SOLUSDT,171.874125


In [ ]:
symbol_prices.reset_index(inplace=True)
symbol_prices

,symbol,PRICE
0,CFXUSDT,0.343060
1,ETHUSDT,3497.434939
2,SOLUSDT,173.478261


In [ ]:
grouped_leader_positions.merge(symbol_prices, left_on='symbol', right_on='symbol')

,symbol,positionAmount_SUM,unrealizedProfit_SUM,cumRealized_SUM,notionalValue_SUM,markPrice_AVERAGE,ABSOLUTE_LEVERED_VALUE_SUM,ABSOLUTE_UNLEVERED_VALUE_SUM,entryPrice_AVERAGE,leverage_AVERAGE,breakEvenPrice_AVERAGE,positionSide,LEVERED_POSITION_SHARE,UNLEVERED_POSITION_SHARE,PRICE
0,CFXUSDT,874.000,1.288879,16.29431,301.489918,0.344954,301.489918,30.148992,0.343479,10.0,0.343651,LONG,0.389573,0.389573,0.343060
1,ETHUSDT,-0.085,1.072634,5497.49111,-298.099316,3507.050780,298.099316,29.809932,3519.670000,10.0,3517.910165,SHORT,0.385191,0.385191,3497.434939
2,SOLUSDT,1.000,0.767267,-317.41150,174.309767,174.309767,174.309767,17.430977,173.542500,10.0,173.703363,LONG,0.225236,0.225236,173.478261


In [ ]:
leader_details

{'userId': 178325873,
 'leadOwner': False,
 'hasCopy': False,
 'leadPortfolioId': '3846188874749232129',
 'nickname': '午后阳光',
 'nicknameTranslate': None,
 'avatarUrl': 'https://public-1306379396.file.myqcloud.com/image/common_notification/20211230/ca5ef4d8-dc73-43ba-9b42-f0ed45929ac5.png',
 'status': 'ACTIVE',
 'description': '只做2倍合约中长线波段，稳定复利，短线勿扰',
 'descTranslate': None,
 'favoriteCount': 3537,
 'currentCopyCount': 147,
 'maxCopyCount': 500,
 'totalCopyCount': 1002,
 'marginBalance': '76392.02987244',
 'initInvestAsset': 'USDT',
 'futuresType': 'UM',
 'aumAmount': '270114.72086068',
 'copierPnl': '25422.50043790',
 'copierPnlAsset': 'USDT',
 'profitSharingRate': '10.00',
 'unrealizedProfitShareAmount': '0',
 'startTime': 1707209121766,
 'endTime': None,
 'closedTime': None,
 'tag': [],
 'positionShow': True,
 'mockCopyCount': 4517,
 'sharpRatio': '6.06915003',
 'hasMock': False,
 'lockPeriod': None,
 'copierLockPeriodTime': None,
 'badgeName': None,
 'badgeModifyTime': None,
 'badge

In [ ]:
transfer_balance = leader_transfer_history["RELATIVE_VALUE"].sum()
historic_PNL = leader_position_history["closingPnl"].sum()
total_balance = transfer_balance + historic_PNL
updated = utils.current_time()

leader = {
    "updated": updated,
    "updated_date": utils.current_readable_time(),
    "details": {
        "updated": updated,
        "data": leader_details
    },
    "positions": filtered_leader_positions.to_dict(),
    "mix": grouped_leader_positions.to_dict(),
    "account": {
        "updated": updated,
        "transfer_balance": transfer_balance,
        "historic_PNL": historic_PNL,
        "total_balance": total_balance,
        "levered_ratio": grouped_leader_positions["ABSOLUTE_LEVERED_VALUE_SUM"].sum() / total_balance,
        "unlevered_ratio": grouped_leader_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"].sum() / total_balance
    },
    "performance": {
        "updated": updated,
        "data": leader_performance

    }
}

leader

{'updated': 1712795768079,
 'updated_date': '2024-04-11 02:36:08',
 'details': {'updated': 1712795768079,
  'data': {'userId': 178325873,
   'leadOwner': False,
   'hasCopy': False,
   'leadPortfolioId': '3846188874749232129',
   'nickname': '午后阳光',
   'nicknameTranslate': None,
   'avatarUrl': 'https://public-1306379396.file.myqcloud.com/image/common_notification/20211230/ca5ef4d8-dc73-43ba-9b42-f0ed45929ac5.png',
   'status': 'ACTIVE',
   'description': '只做2倍合约中长线波段，稳定复利，短线勿扰',
   'descTranslate': None,
   'favoriteCount': 3537,
   'currentCopyCount': 147,
   'maxCopyCount': 500,
   'totalCopyCount': 1002,
   'marginBalance': '76392.02987244',
   'initInvestAsset': 'USDT',
   'futuresType': 'UM',
   'aumAmount': '270114.72086068',
   'copierPnl': '25422.50043790',
   'copierPnlAsset': 'USDT',
   'profitSharingRate': '10.00',
   'unrealizedProfitShareAmount': '0',
   'startTime': 1707209121766,
   'endTime': None,
   'closedTime': None,
   'tag': [],
   'positionShow': True,
   'mockC

In [ ]:
user

{'updated': 1712796459771,
 'updated_date': '2024-04-11 02:47:39',
 'auth': {'updated': 1712795768079,
  'username': 'root',
  'email': 'root@example.com',
  'password_hash': '',
  'binance_api_key': '',
  'binance_secret_hash': ''},
 'details': {'updated': 1712795768079, 'active': True, 'chat_id': 1031182213},
 'account': {'leverage': 5,
  'value_BTC': 0.02853834,
  'value_USDT': 2007.8437344233998,
  'levered_ratio': None,
  'unlevered_ratio': None,
  'collateral_margin_level': 2008.4276214,
  'collateral_value_USDT': 999.0,
  'positions': [{'asset': 'MANTA',
    'free': '0.00741153',
    'locked': '0',
    'borrowed': '0',
    'interest': '0',
    'netAsset': '0.00741153'}],
  'updated': 1712796459771}}

In [ ]:
for leader_id, weight in leaders.items():
    